In [22]:
import cv2
import numpy as np
import pandas as pd 
import scipy
import os
import dimod
from dwave.samplers import SimulatedAnnealingSampler

## Внимание, делаем по следующей статье!
https://www.arxiv-vanity.com/papers/0804.4457/

In [35]:
# Global parameters
S1, S2 = 1, 40
T_feat, T_geom = 0.8, 0.7
# reading the image in grayscale mode
gray = cv2.imread('Data\\760e3945-e0c9-4c9a-b78a-a68fbbee05cd.png', 0)
gray = cv2.GaussianBlur(gray, (5, 5), 0)

In [24]:
lab = pd.read_csv('labeled_ions_team_name_1.csv', sep=';')
load_data = lambda x: cv2.imread('Data\\'+x,0)
lab['sum'] = lab.iloc[:,2:].apply(np.sum, axis=1)
lab_arr = []
for i in lab['Filename']:
    out = load_data(i)
    #out = cv2.medianBlur(out, 3)
    out = cv2.GaussianBlur(out, (5, 5), 0)
    lab_arr.append(out)           

#def find_cent()
def find_points(gray, s1=S1, s2=S2):
    # findcontours 
    # threshold 
    th, threshed = cv2.threshold(gray, 10, 255,  
          cv2.THRESH_BINARY|cv2.THRESH_OTSU) 

    cnts = cv2.findContours(threshed, cv2.RETR_LIST,  
                    cv2.CHAIN_APPROX_SIMPLE)[-2] 

    # filter by area 
    xcnts = [] 
    for cnt in cnts: 
        if s1<cv2.contourArea(cnt) <s2: 
            xcnts.append(cnt)
        
    return xcnts

# Функции для создания вектора признаков из списка массивов
def make_feat_vec(lab_arr, s1=S1):
    res = []
    for j in range(len(lab_arr)):
        matrix = find_points(lab_arr[j], s1=s1)
        out = []
        for i in range(len(matrix)):
           out.append( np.sum(matrix[i], axis=0)/len(matrix[i]) )
        if len(out) == 0:
            res.append(None)
        else:
            res.append(np.vstack(out))
    return res

## Осторожно
Подбор параметра s1 из цикла, который отвечает за области со слишком малой площадью. Как можно увидеть, мы не смогли варируя
только 1 параметр добиться ошибки меньше 1.

In [25]:
hist =[]
s1_array = np.arange (0.1, 20, 0.01)
for i in s1_array:
    out = make_feat_vec(lab_arr, s1=i)
    out = np.array(list(map(len, out)))
    pen = np.sum((lab['sum'] - out)**2)
    hist.append(pen)
a = min(hist)
S1 = s1_array[hist.index(a)]
print(a,'\t',S1)

1.0 	 10.009999999999994


In [26]:
temp = make_feat_vec(lab_arr, s1=S1)
comp_mat = (temp[8] + temp[6] + temp[5])/3
comp_label = [1,2,3,4] #Узнал при помощи ручного сопоставления. Так быстрее!
comp_mat

array([[ 31.1043956 , 133.03296703],
       [ 29.63729604,  94.1995338 ],
       [ 29.66666667,  59.04285714],
       [ 28.44444444,  20.56296296]])

In [27]:
directory = os.fsencode('Data')
file_names = ['Data\\' + os.fsdecode(file) for file in os.listdir(directory)]
lab_arr = []
for i in os.listdir(directory):
    out = load_data(os.fsdecode(i))
    #out = cv2.medianBlur(out, 3)
    out = cv2.GaussianBlur(out, (5, 5), 0)
    lab_arr.append(out)  
    
data_list = make_feat_vec(lab_arr, s1=S1)

In [56]:
# Нормированое скалярное произведение
def dumb_ass(row1, row2):
    return np.dot(row1, row2)/(np.linalg.norm(row1)*np.linalg.norm(row2))

# Граф с ребрами чьи веса равны норме выше  
def make_graph(elem_mat, comp_mat):
    res = np.zeros((len(elem_mat), 4))
    for j in (0,1,2,3):
        for i in range(len(elem_mat)):
            row1 = comp_mat[j]
            row2 = elem_mat[i]
            out = dumb_ass(row1, row2)
            if out > T_feat:       
                res[i,j] = out
    return res

# Проврека на то, что элмент соотвестует
# вершинам с одним обшим индексом из исходного графа
def chek(i, j):
    out = False
    if i in range(j//4*4, j//4*4+4):
        out = True
    return out

# Получение матрицы кубо, где мы сразу делаем матрицу несоотвествия
# и заменяем элементы в соответствии с правилами из статьи
def make_qubo(elem_mat, comp_mat):
    mat = make_graph(elem_mat, comp_mat)
    len1 = len(elem_mat)
    vec = mat.flatten()
    lenin = len(vec)
    vec = vec.reshape(1,-1)
    res = vec.T@vec

    for i in range(lenin):
        for j in range(lenin):
            if i == j:
                res[i,j] = -1
            elif ((res[i,j] < T_geom) or chek(i,j) or chek(j,i)) and res[i,j] !=0:
                res[i,j] = 1
            else:
                res[i,j] = 0
    res[res == 1] = lenin
    return res

''' Недоделанный вариант где мы должны были не умножать евлкидовы нормы для 
каждых элементов матрицы несогласованости, а честно считать нормы заново в 
соотвесвии с векторами трансляции, как было указано в исходной статье 
(наверное, они точно не указали).
'''

def get_indexes (i, j, len1):
    return (i//len1, (j+1)%4, i//len1, j//4)
    
def make_qubo1(elem_mat, comp_mat):
    mat = make_graph(elem_mat, comp_mat)
    len1 = len(elem_mat)
    vec = mat.flatten()
    lenin = len(vec)
    res = np.zeros((lenin, lenin))
    
    for i in range(lenin):
        for j in range(lenin):
            i1, j1, i2, j2, = get_indexes(i, j, len1)
            d1 = dumb_ass(elem_mat[i1], elem_mat[i2])
            d2 = dumb_ass(comp_mat[j1], comp_mat[j2])
            res[i,j] = d1*d2
            if i == j:
                res[i,j] = -1
            elif ((res[i,j] < T_geom) or chek(i,j) or chek(j,i)) and res[i,j] !=0:
                res[i,j] = 1
            else:
                res[i,j] = 0
    res[res == 1] = lenin
    return res
    
#Проверка на симметричность матрицы 
check_elem = make_qubo(data_list[10], comp_mat)
np.all(check_elem==check_elem.T)

True

In [36]:
make_graph(data_list[10], comp_mat)

array([[0.        , 0.80911672, 0.89276411, 0.99992933]])

In [59]:
make_qubo(data_list[10], comp_mat)

array([[-1.,  0.,  0.,  0.],
       [ 0., -1.,  4.,  4.],
       [ 0.,  4., -1.,  4.],
       [ 0.,  4.,  4., -1.]])

In [58]:
get_indexes (4, 4, 16)

(0, 1, 0, 1)

In [63]:
elem = data_list[212]
check_elem = make_qubo(elem, comp_mat)
print(check_elem, '\n\n' ,elem)

[[-1.  0.  0.  0.]
 [ 0. -1.  0.  0.]
 [ 0.  0. -1.  4.]
 [ 0.  0.  4. -1.]] 

 [[28.64705882 20.47058824]]


In [64]:
irb = lambda x: len(x) if not (x is None) else 0
soska = np.array(list(map(irb, data_list)))
np.where(soska == 1)

(array([  9,  10,  44,  54,  80, 145, 160, 212, 222, 273, 299, 305, 339,
        468, 545, 548, 639, 715, 762, 783, 810, 845, 859, 885, 898, 963,
        968, 990, 999], dtype=int64),)

In [65]:
''' У нас получилось множество ответов с одинаковой энергией. По этой причине мы решили собирать статистику для 
нахождения вероятностей
'''
sampler = SimulatedAnnealingSampler()
qubo = check_elem
sampleset = sampler.sample_qubo(qubo, num_reads = 100)
a = list(sampleset.samples())
aboba = lambda x: np.array(list(dict(x).values()))
a = list(map(aboba, a))
a = np.add.reduce(a)
a = a.astype(float)
a /= np.linalg.norm(a)
print(sampleset,'\n\n','Вероятность')

a # вероятности для каждого состояния


    0  1  2  3 energy num_oc.
0   1  1  0  1   -3.0       1
1   1  1  0  1   -3.0       1
2   1  1  0  1   -3.0       1
3   1  1  0  1   -3.0       1
4   1  1  1  0   -3.0       1
5   1  1  0  1   -3.0       1
6   1  1  0  1   -3.0       1
7   1  1  0  1   -3.0       1
8   1  1  1  0   -3.0       1
9   1  1  0  1   -3.0       1
10  1  1  0  1   -3.0       1
11  1  1  1  0   -3.0       1
12  1  1  1  0   -3.0       1
13  1  1  1  0   -3.0       1
14  1  1  1  0   -3.0       1
15  1  1  0  1   -3.0       1
16  1  1  0  1   -3.0       1
17  1  1  0  1   -3.0       1
18  1  1  0  1   -3.0       1
19  1  1  1  0   -3.0       1
20  1  1  0  1   -3.0       1
21  1  1  1  0   -3.0       1
22  1  1  1  0   -3.0       1
23  1  1  1  0   -3.0       1
24  1  1  1  0   -3.0       1
25  1  1  1  0   -3.0       1
26  1  1  0  1   -3.0       1
27  1  1  1  0   -3.0       1
28  1  1  0  1   -3.0       1
29  1  1  1  0   -3.0       1
30  1  1  1  0   -3.0       1
31  1  1  0  1   -3.0       1
32  1  1  

array([0.63245553, 0.63245553, 0.31622777, 0.31622777])

## Рассуждения
Как можно увидеть у нас не получился работоспособный код. Только когда 1 точка, мы получаем, по статистике, что он находит нужный нам элемент (место в битовой строке). Возможные причины мы видим в следующем:
1) Неправильные нормы. Дело в том, что в оригинале статьи не раскрываются какие нормы на каждом этапе используются, а только указывается каким требованиям они должны удовлетворять.
2) Не полное понимание интерпритации полученного бинарного вектора.
3) Мы тупики.

Однако у нас получилось найти точки интереса и сделать из них вектор признаков с погрешностью около 1-2%